In [1]:
%pip install -q -U langchain langchain-google-genai google_generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 421.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [2]:
%pip show langchain

Name: langchain
Version: 0.3.4
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
from google.colab import userdata
import google.generativeai as genai
from google.generativeai import GenerativeModel

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

genai.configure(api_key=GEMINI_API_KEY)

llm: GenerativeModel = GenerativeModel("gemini-1.5-flash")

In [4]:
def get_completion(prompt: str, model=llm) -> str:
  response = model.generate_content(prompt)
  return response.text

get_completion("What is 9 + 1?")

'9 + 1 = 10 \n'

In [5]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [6]:
style = """American English \
in a calm and respectful tone
"""

In [7]:
prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [8]:
res = get_completion(prompt)
res

"I'm quite upset that the lid of my blender flew off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I'd greatly appreciate any advice you can offer. \n"

Gemini using LangChain\
Now doing the same using LangChain.

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm: ChatGoogleGenerativeAI =  ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0, api_key=GEMINI_API_KEY)

In [10]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7cc4bddd0af0>, default_metadata=())

In [11]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [12]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

# print(prompt_template)

# print(prompt_template.messages[0].prompt)

prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [13]:
customer_style = """American English \
in a calm and respectful tone
"""

In [14]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [15]:
customer_message = prompt_template.format_messages(style=customer_style, text=customer_email)

# customer_message
customer_message[0]

HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n", additional_kwargs={}, response_metadata={})

In [16]:
print(type(customer_message))
print(type(customer_message[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [17]:
res = llm.invoke(customer_message)

# res
res.content

"I'm quite frustrated that my blender lid came off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn't cover the cost of cleaning up the mess. I could really use your help right now. \n"

In [18]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [19]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [20]:
service_message = prompt_template.format_messages(style=service_style_pirate, text=service_reply)

# service_message
service_message[0]

HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```\n", additional_kwargs={}, response_metadata={})

In [21]:
res = llm.invoke(service_message)
# res
res.content

"```\nAhoy there, matey!  The warranty, ye see, doesn't cover the cost of cleaning yer kitchen.  It seems ye've gone and misused yer blender, forgetting to secure the lid before starting it up.  Avast, me hearty!  That's a bit of a blunder on yer part.  Best of luck to ye, and fair winds!\n``` \n"

###Output Parsers

In [22]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

In [23]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [24]:
prompt_template = ChatPromptTemplate.from_template(review_template)

In [28]:
message = prompt_template.format_messages(text=customer_review)
message[0].content

"For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\n\n"

In [30]:
llm_response = llm.invoke(message)
llm_response.content

'```json\n{\n"gift": true,\n"delivery_days": 2,\n"price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}\n```'

In [31]:
type(llm_response.content)

str

In [32]:
response.content.get("delivery_days")


NameError: name 'response' is not defined

In [36]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

gift_schema = ResponseSchema(name="gift", description="Was the item purchased as \
                                                       a gift for someone else? \
                                                       Answer True if yes, False \
                                                       if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days", description="How many days did it take \
                                                                         for the product \
                                                                         to arrive? If this information \
                                                                         is not found, output -1.")
price_value_schema = ResponseSchema(name="price_value", description="Extract any sentences about the value or price, and output them as a comma separated Python list.")

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()


In [37]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift": string  // Was the item purchased as                                                        a gift for someone else?                                                        Answer True if yes, False                                                        if not or unknown.\n\t"delivery_days": string  // How many days did it take                                                                          for the product                                                                          to arrive? If this information                                                                          is not found, output -1.\n\t"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.\n}\n```'

In [38]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [39]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [40]:
prompt_template = ChatPromptTemplate.from_template(review_template_2)

In [43]:
message = prompt_template.format_messages(text=customer_review, format_instructions=format_instructions)
message

[HumanMessage(content='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the 

In [47]:
res = llm.invoke(message)
res.content

'```json\n{\n\t"gift": "True",\n\t"delivery_days": "2",\n\t"price_value": "It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."\n}\n```'

In [48]:
output_dict = output_parser.parse(res.content)
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [49]:
type(output_dict)

dict

In [52]:
print(output_dict.get("gift"))
print(output_dict.get("delivery_days"))
print(output_dict.get("price_value"))

True
2
It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.
